In [1]:
%load_ext autoreload
%autoreload 2
from utils import *

In [2]:
df = pd.read_csv("csv/airfoil.csv")

In [3]:
df.sample(5)

,frequency,angle of attack,chord length,velocity,thickness,scaled sound pressure level
435,4000,7.3,0.2286,39.6,0.012348,114.079
692,6300,9.9,0.1524,31.7,0.025278,108.689
1211,400,22.2,0.0254,71.3,0.021418,117.927
693,200,12.6,0.1524,71.3,0.048316,128.354
1458,250,12.3,0.1016,31.7,0.041876,125.857


In [4]:
print("frequency : ", np.unique(df["frequency"].values))
print("angle of attack : ", np.unique(df["angle of attack"].values))
print("chord length : ", np.unique(df["chord length"].values))
print("velocity : ", np.unique(df["velocity"].values))
print("thickness : ", np.unique(df["thickness"].values))
print("scaled sound pressure level : ", np.unique(df['scaled sound pressure level'].values))

frequency :  [  200   250   315   400   500   630   800  1000  1250  1600  2000  2500
  3150  4000  5000  6300  8000 10000 12500 16000 20000]
angle of attack :  [ 0.   1.5  2.   2.7  3.   3.3  4.   4.2  4.8  5.3  5.4  6.7  7.2  7.3
  8.4  8.9  9.5  9.9 11.2 12.3 12.6 12.7 15.4 15.6 17.4 19.7 22.2]
chord length :  [0.0254 0.0508 0.1016 0.1524 0.2286 0.3048]
velocity :  [31.7 39.6 55.5 71.3]
thickness :  [0.00040068 0.00041229 0.00042846 0.00043947 0.00074048 0.00076193
 0.00079182 0.00081216 0.00084863 0.00087322 0.00090748 0.00093079
 0.00121072 0.00131983 0.00142788 0.00146332 0.00150092 0.00152689
 0.0015988  0.00172668 0.00193287 0.00202822 0.00209405 0.00214345
 0.002211   0.00229336 0.00243851 0.00245138 0.00251435 0.00253511
 0.00266337 0.0027238  0.00283081 0.00293031 0.00294804 0.00310138
 0.00313525 0.00331266 0.00336729 0.00346574 0.00372371 0.00392107
 0.00400603 0.00401199 0.00420654 0.00425727 0.0042862  0.0043284
 0.00433288 0.00449821 0.00452492 0.00461377 0.00473801 0.0

In [5]:
df['scaled sound pressure level'].describe()

count    1503.000000
mean      124.835943
std         6.898657
min       103.380000
25%       120.191000
50%       125.721000
75%       129.995500
max       140.987000
Name: scaled sound pressure level, dtype: float64

In [6]:
v_min = df['scaled sound pressure level'].min()

In [7]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

# rank Gini discrimination measure 
g_g = One_minus()
rgdm = Gdm(h, g_g, f_r)

# Gini discrimination measure
gdm = Gdm(h, g_g, f)

# H_M
h_m = Max()
g_m = One_minus_square()
H_m = Gdm(h_m, g_m, f_r)

# H_Q
f_q = Avgdsr()
h_q = Square_root()
H_q = Gdm(h_q, g_g, f_q)

##  7 classes :

In [8]:
v1 = df['scaled sound pressure level'].quantile(1/7)
v2 = df['scaled sound pressure level'].quantile(2/7)
v3 = df['scaled sound pressure level'].quantile(3/7)
v4 = df['scaled sound pressure level'].quantile(4/7)
v5 = df['scaled sound pressure level'].quantile(5/7)
v6 = df['scaled sound pressure level'].quantile(6/7)

In [9]:
df7 = df.copy()
df7.loc[(df7['scaled sound pressure level'] >= v_min) & (df7['scaled sound pressure level'] < v1), 'scaled sound pressure level'] = 7
df7.loc[(df7['scaled sound pressure level'] >= v1) & (df7['scaled sound pressure level'] < v2), 'scaled sound pressure level'] = 6
df7.loc[(df7['scaled sound pressure level'] >= v2) & (df7['scaled sound pressure level'] < v3), 'scaled sound pressure level'] = 5
df7.loc[(df7['scaled sound pressure level'] >= v3) & (df7['scaled sound pressure level'] < v4), 'scaled sound pressure level'] = 4
df7.loc[(df7['scaled sound pressure level'] >= v4) & (df7['scaled sound pressure level'] < v5), 'scaled sound pressure level'] = 3
df7.loc[(df7['scaled sound pressure level'] >= v5) & (df7['scaled sound pressure level'] < v6), 'scaled sound pressure level'] = 2
df7.loc[df7['scaled sound pressure level'] >= v6, 'scaled sound pressure level'] = 1

In [49]:
#df7.drop(['velocity'], axis=1, inplace=True)

In [10]:
#m = 4 # input dimension
m = 5
vectors = df7.values
dataset = LabeledSet(m)
dataset.addExamples(vectors[:,0:m], vectors[:,m:])

In [11]:
for i in range(7):
    print("number of instances of class " + str(i+1) + " : " + str(np.count_nonzero(dataset.y == i+1)))

number of instances of class 1 : 215
number of instances of class 2 : 215
number of instances of class 3 : 214
number of instances of class 4 : 215
number of instances of class 5 : 214
number of instances of class 6 : 215
number of instances of class 7 : 215


In [12]:
NMI1(dataset)

0.01838577616183523

In [13]:
nb_folds = 4

folds = get_folds(dataset, nb_folds)

In [19]:
# Rank Shannon discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rsdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.3118305587817783
standard deviation of accuracy:  0.07793591784903768
average depth :  19.0
standard deviation of depth:  1.8708286933869707
average number of leaves :  171.75
standard deviation of number of leaves:  3.112474899497183
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.5382052595748363
standard deviation of ratio:  0.05135368137232814
average number of pairs used for ratio computing :  47.0
standard deviation of number of pairs:  2.23606797749979


In [20]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(sdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.3037650019357337
standard deviation of accuracy:  0.05798780432071723
average depth :  15.75
standard deviation of depth:  1.299038105676658
average number of leaves :  151.5
standard deviation of number of leaves:  3.774917217635375
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.5623899529433921
standard deviation of ratio:  0.018102828631119117
average number of pairs used for ratio computing :  48.25
standard deviation of number of pairs:  1.920286436967152


In [21]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rgdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.2936830558781778
standard deviation of accuracy:  0.08144929490659568
average depth :  19.5
standard deviation of depth:  1.118033988749895
average number of leaves :  172.5
standard deviation of number of leaves:  3.640054944640259
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.5173722334201171
standard deviation of ratio:  0.014250128013634612
average number of pairs used for ratio computing :  47.0
standard deviation of number of pairs:  1.8708286933869707


In [22]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(gdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.29239256678281067
standard deviation of accuracy:  0.08126707789874103
average depth :  14.25
standard deviation of depth:  1.299038105676658
average number of leaves :  157.5
standard deviation of number of leaves:  4.092676385936225
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.5188824731314149
standard deviation of ratio:  0.045397222716372754
average number of pairs used for ratio computing :  51.25
standard deviation of number of pairs:  3.344772040064913


In [23]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(prdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

KeyboardInterrupt: 

In [ ]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_m, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

In [ ]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_q, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4, 5, 6, 7])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)